In [2]:
import uuid
import asyncio
import asynctnt


In [3]:
PORT = 3303 # check running container

In [1]:
async def get_conn():
    conn = asynctnt.Connection(host="localhost", port=PORT, username='admin', password='pass', reconnect_timeout=None)
    await conn.connect()
    return conn

'''
        :param reconnect_timeout:
                Time in seconds to wait before automatic reconnect
                (set to ``0`` or ``None`` to disable auto reconnect)
'''

In [3]:
space_name = 'users3'
'''box.space.users3:drop()'''

'box.space.users3:drop()'

In [4]:
conn = await get_conn()
data = await conn.select('tester2', [])
tup = data[0]
print('tuple:', tup)

tuple: <TarantoolTuple id=1 band_name='ABBA' year=17>


In [13]:
data = None
try:
    conn = await get_conn()
except ConnectionRefusedError:
    data = "ConnectionRefusedError"
except Exception as exc:
    ...
else:
    data = await conn.execute("select 1 as a, 2 as b")
print(data)

ConnectionRefusedError


In [19]:
'''
box.execute([[CREATE TABLE customers (id INTEGER PRIMARY KEY AUTOINCREMENT, name text);]])
'''
conn = await get_conn()
await conn.execute("insert into customers (name) values (?);", ('James Bond',))
data = await conn.execute('select * from customers')
for row in data:
    print(row)
await conn.disconnect()

<TarantoolTuple ID=1 NAME='James Bond'>
<TarantoolTuple ID=2 NAME='James Bond'>
<TarantoolTuple ID=3 NAME='James Bond'>


In [16]:
async def do_sql(conn: asynctnt.Connection):
    # in console
    '''
    box.execute([[ create table users (
            id int primary key,
            name text
        )    ]])
    '''
    # No AUTOINCREMENT of id here
    try:
        await conn.execute("insert into users (id, name) values (?, ?);", (1, 'James Bond'))
    except asynctnt.exceptions.TarantoolDatabaseError as exc:
        print(str(exc))
        print()

    data = await conn.execute('select * from users')
    for row in data:
        print(row)


conn = await get_conn()
await do_sql(conn)
await conn.disconnect()

Duplicate key exists in unique index 'pk_unnamed_USERS_1' in space 'USERS'

<TarantoolTuple ID=1 NAME='James Bond'>


In [68]:
    # in console
'''  
# -- Create a sequence --
box.schema.sequence.create('id_seq',{min=1, start=1, if_not_exists = true})
or
box.schema.sequence.create('id_name',        { if_not_exists = true })

# -- Create an index that uses the sequence --
box.space.users2:create_index('primary',{ sequence = 'id_seq' })
or
box.space.user2:create_index('primary', {
    parts = { 'id' },
    sequence = 'id_seq',
    unique = true,
    type = 'HASH',
    if_not_exists = true
})


box.schema.space.create('users2')
----is different from: ----
box.execute([[CREATE TABLE users3 (column1 INT PRIMARY key, column2 VARCHAR(10));]])
box.execute([[CREATE TABLE users3 (id INT PRIMARY key, name text);]])
---space and table doesnt work as same...

box.space.user2:insert{nil, 'pen', 123}
box.space.user2:select()
---works for the space which is created by (box.schema.space.create('users2'))
'''


In [17]:
conn = await get_conn()
# data = await conn.execute('select * from goods')
# print(data)
await conn.disconnect()

<Response sync=4 rowcount=0 data=[]>


In [13]:
table_name = 'users5'
'''in console
box.execute([[CREATE TABLE users5 (id INTEGER PRIMARY KEY AUTOINCREMENT, name text);]])
'''
conn = await get_conn()
await conn.execute("insert into users5 (name) values (?);", ('James Bond',))

data = await conn.execute(f'select * from {table_name}')
print(data)
await conn.disconnect()

<Response sync=6 rowcount=3 data=[<TarantoolTuple ID=1 NAME='James Bond'>, <TarantoolTuple ID=2 NAME='James Bond'>, <TarantoolTuple ID=3 NAME='James Bond'>]>


In [ ]:
'''
box.execute([[CREATE TABLE TESTER4 (id INTEGER PRIMARY KEY AUTOINCREMENT, name text);]])
box.space.TESTER4:insert{2, "w"}
box.space.TESTER4:select()

'''

In [13]:
conn = await get_conn()
# await conn.execute("insert into TESTER4 (name) values (?);", ('James Bond3',))
data = await conn.execute('select * from SEQSCAN  APP order by YEAR desc limit 30;')
for row in data:
    print(row)
await conn.disconnect()

In [ ]:
# TODO (not checked)
# box.schema.sequence.create('id_seq',{min=1, start=1, if_not_exists = true})
# box.schema.space.create('user4', {
#     format = {
#         {name = 'id', type = 'unsigned' },
#         {name = 'name', type = 'string' },
#         {name = 'code', type = 'unsigned'},
#     },
#     if_not_exists = true
# })

# box.schema.sequence.create('user4_id',  { if_not_exists = true })
# box.space.users4:create_index('primary',{ parts = { 'id' },
#         sequence = 'user4_id',
#         unique = true,
#         type = 'HASH',
#         if_not_exists = true})